# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.count()
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bazarak,35.3129,69.5152,5.36,50,100,1.58,AF,1701207880
1,1,shambu,9.5667,37.1000,10.46,79,16,1.13,ET,1701207880
2,2,pisco,-13.7000,-76.2167,22.03,73,0,11.83,PE,1701207420
3,3,la cruz,23.9167,-106.9000,28.26,45,1,5.73,MX,1701207755
4,4,ushuaia,-54.8000,-68.3000,6.81,81,75,12.35,AR,1701207403


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1,
    color = 'City'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
idealcities = city_data_df[(city_data_df['Max Temp'] < 40) &
                             (city_data_df['Max Temp'] > 20) &
                             (city_data_df['Wind Speed'] < 15) &
                             (city_data_df['Humidity'] < 80) &
                             (city_data_df['Cloudiness'] <= 20)
                           ]
#drop null values

# Drop any rows with null values
idealcities = idealcities.dropna()
#idealcities.count()
# Display sample data
# YOUR CODE HERE
idealcities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,pisco,-13.7000,-76.2167,22.03,73,0,11.83,PE,1701207420
3,3,la cruz,23.9167,-106.9000,28.26,45,1,5.73,MX,1701207755
14,14,arica,-18.4750,-70.3042,23.75,64,0,5.14,CL,1701207883
28,28,nouadhibou,20.9310,-17.0347,20.99,49,0,4.12,MR,1701207505
66,66,jiwani,25.0500,61.7417,24.59,58,0,3.44,PK,1701207893


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
col_to_keep = ['City','Country','Lat','Lng','Humidity']
hotel_df = idealcities[col_to_keep].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,pisco,PE,-13.7000,-76.2167,73,
3,la cruz,MX,23.9167,-106.9000,45,
14,arica,CL,-18.4750,-70.3042,64,
28,nouadhibou,MR,20.9310,-17.0347,49,
66,jiwani,PK,25.0500,61.7417,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "apiKey": geoapify_key,
     "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
     
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
la cruz - nearest hotel: Hotel Las Palmas
arica - nearest hotel: Hotel Andalucía 
nouadhibou - nearest hotel: Hotel valencia
jiwani - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
charters towers - nearest hotel: No hotel found
hilo - nearest hotel: Dolphin Bay Hotel
vallenar - nearest hotel: Hotel del Marqués
lauro de freitas - nearest hotel: Malibu Plaza Hotel
omdurman - nearest hotel: فندق مزدلفة السياحي
nova sintra - nearest hotel: Residência Ka Dencho
kiffa - nearest hotel: فندق كيفة
jesus maria - nearest hotel: Hotel Jesús María
santiago - nearest hotel: apart arturo prat
hawaiian paradise park - nearest hotel: No hotel found
puli - nearest hotel: 金山大飯店
ebaye - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
howrah - nearest hotel: Sun India Guest House
santa rosa de cabal - nearest hotel: Hotel Cabal
al hazm - nearest hotel: No hotel found
lautoka - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
2,pisco,PE,-13.7000,-76.2167,73,La Portada
3,la cruz,MX,23.9167,-106.9000,45,Hotel Las Palmas
14,arica,CL,-18.4750,-70.3042,64,Hotel Andalucía
28,nouadhibou,MR,20.9310,-17.0347,49,Hotel valencia
66,jiwani,PK,25.0500,61.7417,58,No hotel found
71,port mathurin,MU,-19.6833,63.4167,73,Escale Vacances
95,charters towers,AU,-20.1000,146.2667,60,No hotel found
120,hilo,US,19.7297,-155.0900,75,Dolphin Bay Hotel
132,vallenar,CL,-28.5708,-70.7581,33,Hotel del Marqués
140,lauro de freitas,BR,-12.8944,-38.3272,78,Malibu Plaza Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = 'City',
    hover_cols=["City", "Country", "Hotel Name"],
    title="Cities and Nearest Hotels"
)


# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)